In [1]:
import random

In [2]:
class Cell:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        self.is_mine = False
        self.is_revealed = False
        self.is_flagged = False
        self.neighbor_mines = 0

    def __str__(self):
        s = "F " if self.is_flagged else ". "

        if self.is_revealed:
            s = '* ' if self.is_mine else str(self.neighbor_mines) + " "
        return s

In [118]:
class GameField:
    def __init__(self, width, height, number_of_bombs):
        self.width = width
        self.height = height
        self.is_playing = True
        self.win = False
        self.bombs_left = number_of_bombs
        self.field = [[Cell(x, y) for y in range(self.height)] for x in range(self.width)]
        self.place_bombs_randomly(number_of_bombs)
        self.calculate_adjacent_numbers()
        
    def __str__(self):
        s = ""
        for y in range(self.height):
            for x in range(self.width):
                s += str(self.field[x][y])
            s += "\n"
        return s
    
    def is_within_bounds(self, x, y):
        return x >= 0 and y >= 0 and x < self.width and y < self.height 
    
    def get_adjacent_cells(self, x, y):
        lst = []
        for i in [-1, 0, 1]:
            for j in [-1, 0, 1]:
                if self.is_within_bounds(x+i,y+j) and not (i==0 and j==0):
                    lst.append(self.field[x+i][y+j])
        return lst
    
    def place_bombs_randomly(self, number_of_bombs):
        cells_shuffle_lst = [cell for row in self.field for cell in row]
        random.shuffle(cells_shuffle_lst)
        for random_cell in cells_shuffle_lst[:number_of_bombs]:
            self.field[random_cell.x][random_cell.y].is_mine = True
    
    
    def calculate_adjacent_numbers(self):
        for row in self.field:
            for cell in row:
                adj_mines = [adj_cell for adj_cell in self.get_adjacent_cells(cell.x, cell.y) \
                             if adj_cell.is_mine == True]
                cell.neighbor_mines = len(adj_mines)
    
    def reveal_all_bombs(self):
        for row in self.field:
            for cell in row:
                if cell.is_mine: cell.is_revealed = True
                    
    
    def check_win_condition(self):
        check_lst = []
        for row in self.field:
            for cell in row:
                check_lst.append((cell.is_mine and cell.is_flagged) or (not cell.is_mine and cell.is_revealed))    
        if all(check_lst):
            self.win = True
            self.is_playing = False

    def toggle_flag(self, x, y):
        if self.is_within_bounds(x, y):
            cell = self.field[x][y]   
        
            if cell.is_flagged:
                cell.is_flagged = False
                self.bombs_left += 1
            elif self.bombs_left > 0 and not cell.is_revealed:
                cell.is_flagged = True
                self.bombs_left -= 1
            self.check_win_condition()
    
    def reveal_cell(self, x, y):
        if self.is_within_bounds(x, y):
            cell = self.field[x][y]

            if not cell.is_revealed and not cell.is_flagged:
                if cell.is_mine:
                    self.reveal_all_bombs()                
                    self.is_playing = False              
                else:
                    cell.is_revealed = True
                    if cell.neighbor_mines == 0:
                        for adj_cell in self.get_adjacent_cells(cell.x, cell.y):
                            self.reveal_cell(adj_cell.x, adj_cell.y)
                    self.check_win_condition()


In [189]:
game = GameField(width = 25, height = 25, number_of_bombs = 35)

In [192]:
print(game)
print(f"playing: {game.is_playing}, bombs left: {game.bombs_left}, win: {game.win}")

0 1 . 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 . . . 
0 1 1 2 1 2 1 1 0 0 0 1 1 1 0 0 0 0 1 1 1 1 . . . 
0 0 0 1 . . . 1 0 1 1 2 . 1 0 0 0 0 1 . . . . . . 
0 0 0 1 1 2 1 1 0 2 . 3 1 1 0 0 0 0 1 1 2 . . . . 
0 0 0 0 0 0 0 0 0 2 . 2 0 0 0 0 0 0 0 0 1 . . . . 
0 0 0 0 0 0 0 0 0 2 . 2 0 0 0 0 0 0 0 0 1 1 1 1 . 
1 1 1 0 0 0 0 0 0 1 . 1 0 0 0 0 1 1 2 1 1 0 0 1 1 
. . 1 0 0 0 0 0 0 1 . 1 0 0 0 0 1 . . . 1 0 0 0 0 
1 1 1 1 1 1 0 0 0 1 . 1 0 0 0 0 1 1 2 . 1 0 1 1 1 
0 0 0 2 . 2 0 0 0 1 . 1 0 0 0 0 0 0 1 . 1 0 1 . . 
0 0 0 2 . 2 0 0 0 1 . 1 0 0 0 0 1 1 2 . 1 0 1 2 . 
0 0 0 1 1 1 1 1 1 1 . 1 1 1 1 0 1 . 2 1 1 0 0 1 1 
0 0 0 0 0 0 1 . . . . . . . 1 0 1 1 1 0 0 0 0 0 0 
0 0 0 0 0 0 1 . 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 1 . 1 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 
0 0 1 1 1 0 1 . 2 1 1 0 0 1 . 1 0 0 0 0 0 0 0 0 0 
0 0 1 . 1 0 1 1 2 . 1 0 0 1 1 1 0 0 0 0 0 0 0 0 0 
0 0 1 1 1 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
1 1 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 
. 2 0 0 0 0 0 0 0 0 0 1 . 1 0 0

In [191]:
game.reveal_cell(0, 0)

In [181]:
game.toggle_flag(0, 3)